# Clean GNN Model with AttentiveFP

**Problem**: CV-LB gap has intercept (0.0525) > target (0.0347). Current approach CANNOT reach target.

**Solution**: GNN learns from molecular STRUCTURE, not IDENTITY. Can generalize to unseen solvents.

**Key**: This notebook has EXACTLY 3 submission cells at the end (no extra cells).

In [3]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch Geometric imports
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool

# RDKit imports
from rdkit import Chem

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.float32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [5]:
# Load SMILES lookup
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
print(f'SMILES lookup: {len(SMILES_DF)} solvents')

SMILES lookup: 26 solvents


In [6]:
# SMILES to molecular graph conversion
def smiles_to_graph(smiles):
    """Convert SMILES to PyTorch Geometric Data object."""
    if '.' in smiles:
        smiles = smiles.split('.')[0]
    
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Could not parse SMILES: {smiles}")
    
    # Atom features (6 features)
    atom_features = []
    for atom in mol.GetAtoms():
        features = [
            atom.GetAtomicNum(),
            atom.GetDegree(),
            atom.GetFormalCharge(),
            int(atom.GetHybridization()),
            int(atom.GetIsAromatic()),
            atom.GetTotalNumHs(),
        ]
        atom_features.append(features)
    
    x = torch.tensor(atom_features, dtype=torch.float)
    
    # Edge index
    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.extend([[i, j], [j, i]])
    
    if len(edge_index) == 0:
        edge_index = [[0, 0]]
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    
    return Data(x=x, edge_index=edge_index)

# Pre-compute all graphs
SOLVENT_GRAPHS = {}
for solvent_name in SMILES_DF.index:
    smiles = SMILES_DF.loc[solvent_name, 'solvent smiles']
    SOLVENT_GRAPHS[solvent_name] = smiles_to_graph(smiles)

print(f'Pre-computed {len(SOLVENT_GRAPHS)} solvent graphs')

Pre-computed 26 solvent graphs


In [7]:
# Simple GNN Model using GCNConv
class SimpleGNN(nn.Module):
    def __init__(self, node_dim=6, hidden_dim=64, output_dim=32):
        super().__init__()
        self.conv1 = GCNConv(node_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.relu(self.conv1(x, edge_index))
        x = self.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

# Combined GNN + MLP Model for single solvent
class GNNModel(nn.Module):
    def __init__(self, gnn_output_dim=32, kinetics_dim=5, hidden_dim=64):
        super().__init__()
        self.gnn = SimpleGNN(output_dim=gnn_output_dim)
        self.mlp = nn.Sequential(
            nn.Linear(gnn_output_dim + kinetics_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, 3),
            nn.Sigmoid()
        )
        
    def forward(self, graph_batch, kinetics):
        gnn_emb = self.gnn(graph_batch)
        combined = torch.cat([gnn_emb, kinetics], dim=1)
        return self.mlp(combined)

# Combined GNN + MLP Model for mixed solvents
class GNNModelMixed(nn.Module):
    def __init__(self, gnn_output_dim=32, kinetics_dim=5, hidden_dim=64):
        super().__init__()
        self.gnn = SimpleGNN(output_dim=gnn_output_dim)
        self.mlp = nn.Sequential(
            nn.Linear(gnn_output_dim * 2 + kinetics_dim + 1, hidden_dim),  # +1 for pct
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, 3),
            nn.Sigmoid()
        )
        
    def forward(self, graph_batch_a, kinetics, pct, graph_batch_b):
        gnn_emb_a = self.gnn(graph_batch_a)
        gnn_emb_b = self.gnn(graph_batch_b)
        # Weighted combination based on percentage
        pct_expanded = pct.unsqueeze(1)
        combined = torch.cat([gnn_emb_a, gnn_emb_b, kinetics, pct_expanded], dim=1)
        return self.mlp(combined)

print('GNN models defined')

GNN models defined


In [8]:
# GNN Wrapper class for training and prediction
class GNNModelWrapper:
    def __init__(self, data='single', n_models=3):
        self.data_type = data
        self.n_models = n_models
        self.models = []
        
    def _get_kinetics(self, X):
        """Extract kinetics features."""
        time_m = X["Residence Time"].values.reshape(-1, 1)
        temp_c = X["Temperature"].values.reshape(-1, 1)
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        return np.hstack([time_m, temp_c, inv_temp, log_time, interaction])
    
    def _get_graphs(self, solvent_names):
        """Get pre-computed graphs for solvents."""
        return [SOLVENT_GRAPHS[name].clone() for name in solvent_names]
    
    def train_model(self, X_train, Y_train, epochs=100):
        Y_np = Y_train.values if hasattr(Y_train, 'values') else Y_train
        kinetics = torch.tensor(self._get_kinetics(X_train), dtype=torch.float32).to(device)
        targets = torch.tensor(Y_np, dtype=torch.float32).to(device)
        
        if self.data_type == 'single':
            graphs = self._get_graphs(X_train["SOLVENT NAME"].values)
        else:
            graphs_a = self._get_graphs(X_train["SOLVENT A NAME"].values)
            graphs_b = self._get_graphs(X_train["SOLVENT B NAME"].values)
            pct = torch.tensor(X_train["SolventB%"].values, dtype=torch.float32).to(device)
        
        self.models = []
        for seed in range(self.n_models):
            torch.manual_seed(42 + seed)
            
            if self.data_type == 'single':
                model = GNNModel().to(device)
            else:
                model = GNNModelMixed().to(device)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            
            model.train()
            batch_size = 32
            n_samples = len(Y_np)
            
            for epoch in range(epochs):
                indices = torch.randperm(n_samples)
                for start in range(0, n_samples, batch_size):
                    end = min(start + batch_size, n_samples)
                    batch_idx = indices[start:end]
                    
                    batch_kinetics = kinetics[batch_idx]
                    batch_y = targets[batch_idx]
                    
                    if self.data_type == 'single':
                        batch_graphs = [graphs[i] for i in batch_idx]
                        batch_graph = Batch.from_data_list(batch_graphs).to(device)
                        pred = model(batch_graph, batch_kinetics)
                    else:
                        batch_graphs_a = [graphs_a[i] for i in batch_idx]
                        batch_graphs_b = [graphs_b[i] for i in batch_idx]
                        batch_graph_a = Batch.from_data_list(batch_graphs_a).to(device)
                        batch_graph_b = Batch.from_data_list(batch_graphs_b).to(device)
                        batch_pct = pct[batch_idx]
                        pred = model(batch_graph_a, batch_kinetics, batch_pct, batch_graph_b)
                    
                    optimizer.zero_grad()
                    loss = criterion(pred, batch_y)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            self.models.append(model)
    
    def predict(self, X_test):
        kinetics = torch.tensor(self._get_kinetics(X_test), dtype=torch.float32).to(device)
        
        if self.data_type == 'single':
            graphs = self._get_graphs(X_test["SOLVENT NAME"].values)
            graph_batch = Batch.from_data_list(graphs).to(device)
        else:
            graphs_a = self._get_graphs(X_test["SOLVENT A NAME"].values)
            graphs_b = self._get_graphs(X_test["SOLVENT B NAME"].values)
            graph_batch_a = Batch.from_data_list(graphs_a).to(device)
            graph_batch_b = Batch.from_data_list(graphs_b).to(device)
            pct = torch.tensor(X_test["SolventB%"].values, dtype=torch.float32).to(device)
        
        preds = []
        with torch.no_grad():
            for model in self.models:
                if self.data_type == 'single':
                    pred = model(graph_batch, kinetics)
                else:
                    pred = model(graph_batch_a, kinetics, pct, graph_batch_b)
                preds.append(pred.cpu())
        
        return torch.clamp(torch.stack(preds).mean(dim=0), 0, 1).double()

print('GNNModelWrapper defined')

GNNModelWrapper defined


In [9]:
# Quick test on single fold
X_single, Y_single = load_data("single_solvent")
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

print(f"Test solvent: {test_solvent}")
print(f"Training samples: {mask.sum()}, Test samples: {(~mask).sum()}")

model = GNNModelWrapper(data='single', n_models=1)
model.train_model(X_single[mask], Y_single[mask], epochs=50)
preds = model.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse = np.mean((actuals - preds.numpy()) ** 2)
print(f'Test fold MSE: {mse:.6f}')

Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol
Training samples: 619, Test samples: 37


Test fold MSE: 0.057851


In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GNNModelWrapper(data='single', n_models=2)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, epochs=50)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

# Calculate single solvent MSE
actuals_single = []
for solvent in sorted(X["SOLVENT NAME"].unique()):
    mask = X["SOLVENT NAME"] == solvent
    actuals_single.append(Y[mask].values)
actuals_single = np.vstack(actuals_single)
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:06,  6.41s/it]

2it [00:12,  6.43s/it]

3it [00:19,  6.33s/it]

4it [00:25,  6.26s/it]

...

22it [02:21,  6.47s/it]

23it [02:28,  6.47s/it]

24it [02:34,  6.49s/it]

24it [02:34,  6.44s/it]


Single Solvent MSE: 0.015595


In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GNNModelWrapper(data='full', n_models=2)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, epochs=50)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

# Calculate full data MSE
actuals_full = []
ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y[mask].values)
actuals_full = np.vstack(actuals_full)
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')

# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)
print(f'\nOverall MSE: {overall_mse:.6f}')
print(f'Baseline (exp_030): CV 0.008298')

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:18, 18.13s/it]

2it [00:36, 18.25s/it]

3it [00:55, 18.50s/it]

4it [01:13, 18.46s/it]

...

11it [03:25, 19.20s/it]

12it [03:44, 19.36s/it]

13it [04:04, 19.52s/it]

13it [04:04, 18.83s/it]


Full Data MSE: 0.031024

Overall MSE: 0.025649
Baseline (exp_030): CV 0.008298


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################